# Setup

In [ ]:
def double_delayed(x):
    import time
    # simulate a function that takes a bit of time
    time.sleep(1)
    return x * 2

# Existing Model

In [ ]:
import time
from funcx.sdk.client import FuncXClient
from funcx.utils.errors import TaskPending
from double_delayed import double_delayed

fxc = FuncXClient()

# tutorial endpoint
ep_id = '4b116d3c-1703-4f8f-9f6f-39921e5864df'
func_id = fxc.register_function(double_delayed)

x = 50
task_id = fxc.run(x, endpoint_id=ep_id, function_id=func_id)

while True:
    try:
        # HTTP task query
        result = fxc.get_result(task_id)
        print(result)
        break
    except TaskPending:
        # task is still pending, continue waiting
        print('Task pending')
    time.sleep(1)

# Async Example

In [ ]:
from funcx.sdk.client import FuncXClient
from double_delayed import double_delayed

fxc = FuncXClient(asynchronous=True)
# tutorial endpoint
ep_id = '4b116d3c-1703-4f8f-9f6f-39921e5864df'
func_id = fxc.register_function(double_delayed)

async def task():
    x = 50
    result = await fxc.run(x, endpoint_id=ep_id, function_id=func_id)
    print(result)

await task()

# Executor Example

In [ ]:
from funcx import FuncXClient
from funcx.sdk.executor import FuncXExecutor
from double_delayed import double_delayed

fxc = FuncXClient()
fx = FuncXExecutor(fxc)

# tutorial endpoint
ep_id = '4b116d3c-1703-4f8f-9f6f-39921e5864df'

x = 50
future = fx.submit(double_delayed, x, endpoint_id=ep_id)
result = future.result()
print(result)

# Executor Batch Example

In [ ]:
import random
import timeit
from funcx import FuncXClient
from funcx.sdk.executor import FuncXExecutor
from double_delayed import double_delayed

fxc = FuncXClient()
fx = FuncXExecutor(fxc, batch_enabled=False, batch_interval=1.0)

# tutorial endpoint (switch to your own endpoint to run much faster)
ep_id = '4b116d3c-1703-4f8f-9f6f-39921e5864df'

def run():
    futures = []
    for _ in range(50):
        x = random.randint(0, 100)
        future = fx.submit(double_delayed, x, endpoint_id=ep_id)
        futures.append(future)

    for future in futures:
        result = future.result()
        print(f'Result: {result}')

t = timeit.timeit(run, number=1)
print(f'Time: {round(t, 2)}s')